In [25]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from os import listdir
from os.path import isfile, join
import sys
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as colors
import numpy as np
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
#import matplotlib
from PIL import Image, ImageDraw
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from matplotlib import pyplot
from shapely.geometry.polygon import LinearRing, Polygon
# from multiprocessing import Pool
import multiprocessing
import time
import random
from scipy.spatial import ConvexHull
sys.path.insert(0, os.path.dirname(os.path.abspath('.'))) # to make imports relative to project root work


In [6]:
#load df lables from pickle
df = pd.read_pickle("./labels.pkl")
df_polygon = df[df['annotation_type'] == 'Polygon']

df_polygon


,?,cell,comment,annotation_type,??,position,filename,txt_path,img_path
0,1-1,Uninfected,No_comment,Polygon,17,"[(3199, 2214), (3216, 2216), (3232, 2218), (32...",IMG_20151106_112252,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...
1,1-2,Uninfected,No_comment,Polygon,16,"[(3193, 2345), (3210, 2341), (3228, 2343), (32...",IMG_20151106_112252,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...
2,1-3,Uninfected,No_comment,Polygon,18,"[(3088, 2134), (3105, 2135), (3131, 2145), (31...",IMG_20151106_112252,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...
3,1-4,Uninfected,No_comment,Polygon,16,"[(3053, 2267), (3072, 2269), (3092, 2274), (31...",IMG_20151106_112252,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...
4,1-5,Uninfected,No_comment,Polygon,19,"[(3342, 2261), (3360, 2260), (3380, 2269), (34...",IMG_20151106_112252,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/306C2...
...,...,...,...,...,...,...,...,...,...
166,4-167,Parasitized,No_comment,Polygon,25,"[(2399, 383), (2404, 363), (2412, 351), (2426,...",IMG_20151004_131030,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...
167,4-168,Parasitized,No_comment,Polygon,21,"[(3522, 2250), (3536, 2230), (3563, 2215), (35...",IMG_20151004_131030,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...
168,4-169,Parasitized,No_comment,Polygon,20,"[(3321, 2138), (3335, 2114), (3347, 2103), (33...",IMG_20151004_131030,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...
169,4-170,Parasitized,No_comment,Polygon,20,"[(3433, 1867), (3441, 1845), (3455, 1831), (34...",IMG_20151004_131030,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...,../NIH-NLM-ThinBloodSmearsPf/Polygon Set/276C1...


In [23]:
#df_polygon[df_polygon['position']]

#df_polygon.img_path[:1]
#df_polygon.filename[:1]
df_polygon.position[:1]


0    [(3199, 2214), (3216, 2216), (3232, 2218), (32...
Name: position, dtype: object

In [27]:
def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    # XXX both work
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
#     rotations = np.vstack([
#         np.cos(angles),
#         -np.sin(angles),
#         np.sin(angles),
#         np.cos(angles)]).T
    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)

    return rval

In [51]:


#data = {"type":"FeatureCollection",
# "features":[{"type":"Feature",
#              "properties":{"osm_id":"1269601",
#                            "leisure":"pitch",
#                            "type":"multipolygon",
#                            "sport":"soccer"},
#              "geometry":{
#                  "type":"Polygon",
#                  "coordinates":[[
#                      [6.6131123,46.5124914],
#                      [6.6129421,46.5125385],
#                      [6.6127998,46.5125783],
#                      [6.6126291,46.512626],
#                      [6.6125308,46.5124593],
#                      [6.6127016,46.5124121],
#                      [6.6128452,46.5123724],
#                      [6.6130153,46.5123244],
#                      [6.6131123,46.5124914]]
#                  ]
#              }
#             }
#            ]
#      }

# polygons can have holes, so, ["coordinates"][0] gives you boundary of polygon.
# If you have multipolygon, ["coordinates"][0][0] gives you the first polygon boundary.
#geom = data["features"][0]["geometry"]["coordinates"][0]

#mabr = minimum_bounding_rectangle(np.array(geom))

# OUT:
#array[[  6.6131123 ,  46.5124914 ],
#      [  6.61306213,  46.51231129],
#      [  6.6125308 ,  46.5124593 ],
#      [  6.61258097,  46.51263941]]

#data2 = dict(data) # copy data to data2    
#data2["features"][0]["geometry"]["coordinates"][0] = mabr.tolist()

polygon = df_polygon.position[:1]
rect = minimum_bounding_rectangle(poly_arr)

ValueError: setting an array element with a sequence.

In [50]:
poly_arr = polygon.array
poly_arr

<PandasArray>
[[(3199, 2214), (3216, 2216), (3232, 2218), (3245, 2226), (3260, 2243), (3266, 2267), (3263, 2298), (3249, 2321), (3227, 2331), (3186, 2329), (3167, 2320), (3151, 2304), (3139, 2282), (3139, 2257), (3148, 2239), (3164, 2224), (3181, 2217)]]
Length: 1, dtype: object